In [28]:
import torch # ML framework: It use tensor DS for math and calculus
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Hyperpara
block_size = 8
batch_size = 4 # How many blocks we process in ||
max_iters = 10000
eval_interval = 2500
learning_rate = 3e-4

cuda


In [29]:
with open('data/magic_of_oz.txt', 'r', encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
vocab_size = len(chars) # How many unique chars in dataset
print(chars)
print(vocab_size)



['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
80


In [30]:
# tokenization -> encoding/deconding encoding convert each char to int

string_to_int = { char:index for index,char in enumerate(chars) }
int_to_string = { index:char for index,char in enumerate(chars) }
encode = lambda string: [string_to_int[char] for char in string]
decode = lambda int_list: ''.join([int_to_string[int] for int in int_list])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])



tensor([79,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 50, 33, 64, 64, 73, 71,
        72, 70, 53, 72, 61, 67, 66, 51,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1, 44, 60, 57,  1, 30, 53, 65, 67, 73, 71,  1, 39, 78,  1, 26,
        67, 67, 63, 71,  0,  0,  0, 43, 61, 66])


In [31]:
# Validation and training split
n = int(0.08 * len(data))
train_data = data[:n]
validation_data = data[n:]


In [32]:
def get_batch(split):
    data = train_data if split == 'train' else validation_data
    ix = torch.randint(0, len(data) - block_size, (batch_size, ))
    print(f"Random indices: {ix}")
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # Push data to GPU means its calc handled by GPU
    return x, y

x, y = get_batch('train')
print('Inputs:')
print(x)
print('Target:')
print(y)

Random indices: tensor([ 4414, 10835,  8876, 16559])
Inputs:
tensor([[ 1, 77, 67, 73,  1, 72, 60, 53],
        [70,  1, 65, 53, 66, 77,  1, 77],
        [60, 57,  0, 75, 67, 70, 64, 56],
        [67, 70, 57, 56,  1, 60, 61, 65]], device='cuda:0')
Target:
tensor([[77, 67, 73,  1, 72, 60, 53, 72],
        [ 1, 65, 53, 66, 77,  1, 77, 57],
        [57,  0, 75, 67, 70, 64, 56,  1],
        [70, 57, 56,  1, 60, 61, 65,  0]], device='cuda:0')


In [33]:
# Bigram example
x = train_data[: block_size]
y = train_data[1: block_size+1]

for t in range(block_size):
    context = x[: t+1]
    target = y[t]
    print(f"When input is {context} target is {target}")

When input is tensor([79]) target is 1
When input is tensor([79,  1]) target is 1
When input is tensor([79,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1,  1,  1]) target is 1


In [34]:
# nn.Module is a fundamental building block used to create neural network architectures. It's a base class for all neural network modules.
# provides methods for various functionalities like managing parameters, defining layers, and executing forward propagation.
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:
            B, T, C = logits.shape # Batch, Time, Channels is vocab_size
            logits = logits.view(B*T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index) # Logits are the raw scores or probabilities that a model assigns to each class
            
            logits = logits[:, -1, :] # On last time step became (B, C)
            probs = F.softmax(logits, dim=-1) # Apply softmax for probablities
            index_next = torch.multinomial(probs, num_samples=1) # Sample from distribution (B, 1)
            index = torch.cat((index, index_next), dim=1) # Append sample index to running index (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device) # torch.long in int64 integer
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
generated_chars

'\n_b\ufeffM?73n9r_vSVEo)c:)IO(Rnc!uyfnPqUIC4c81ZCDk&iPox5-?\n:1_LIdJ:a3VN\'ne2Dv-Z\n  YUD,k&z8FBPAC:;,Id"qyqJFPS.Z(8H3;!]nrx-4Yy03;d&29-&3W[FPAz*KV 5won6\'fMpB(G-&)F7gPrpxf\ufeff6mqn-rVnUkc]rgA;7( kK-6k][2WZ]73I0JwVA;CFafiWc84\'fmL.J\n8GbA;B:JBg:L\ufeffPbuu?CqC\'h\ng zr\ufeffk&SViror"(v68E([z.t(PKCEtevR;IILtK;5PG-VZYpnT_x9]\n7FhQ-C4G,;9\n(LAf 6qy UzkQ)(KfhoydO*E"(5G6D,\ufeffPQ1n_zYQ3.n6W[TfmEM!V\n8HOh\nkGTW\ufeff3CD,]UIoqgu!MJrC1WGT\ufeffUI9G-tA\n:dQsrdA25d*6\'s)IW\ufeff3Ff[3zkFO(kGbod;daGImEGIoyAqwjys&RpKKA3W3.D0smFN" ViiVIo?\ufeff1IoKQ1f[CNqG-ZYl[58*J[N'

In [ ]:
# Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')

    # Eval loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # It reset prev gradients so they do not add with current gradient
    loss.backward()
    optimizer.step()
print(loss.item())


Random indices: tensor([13395,  4055,  4084, 16447])
Random indices: tensor([ 7826,  9695, 11816,  6454])
Random indices: tensor([ 2530, 15187, 13177, 18250])
Random indices: tensor([ 1052, 14291,  2998, 14009])
Random indices: tensor([ 2439, 12658,  8720,  3833])
Random indices: tensor([ 9401,   837, 13411, 18398])
Random indices: tensor([ 6057, 11453, 13459,  9859])
Random indices: tensor([15919,  3222, 14077,   692])
Random indices: tensor([ 2438,  1737,  2886, 11645])
Random indices: tensor([14116, 10468,  9906,  9593])
Random indices: tensor([  256, 18012,  1784, 15502])
Random indices: tensor([ 5165,  2227, 16535, 14088])
Random indices: tensor([ 1627, 11145, 10833,  1913])
Random indices: tensor([13182,  9602, 11083, 11322])
Random indices: tensor([8505, 3389,  680, 7127])
Random indices: tensor([15339, 10762, 16359, 14423])
Random indices: tensor([ 8921,  3500, 15359,  5306])
Random indices: tensor([ 4030,  2117,  1351, 15712])
Random indices: tensor([ 1793,  6424,   474, 14304

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
generated_chars